In [1]:
from QASMBench.interface.qiskit import QASMBenchmark

# path to the root directory of QASMBench
path = "QASMBench"
# selected category for QASMBench
category = "large" 
# select only the circuits with the number of qubits in the list
num_qubits_list = list(range(1, 101))
# whether to remove the final measurement in the circuit
remove_final_measurements = True
# whether to use qiskit.transpile() to transpile the circuits (note: must provide qiskit backend)
do_transpile = True  # Changed to True for transpiling circuits
# arguments for qiskit.transpile(). backend should be provided at least
transpile_args = { 'basis_gates': ['u', 'cp']}
bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, do_transpile=do_transpile, **transpile_args, remove_final_measurements=remove_final_measurements)

# circ_list = bm.circ_list
circ_name_list = bm.circ_name_list
for name in circ_name_list:
    print(name)

adder_n28
qft_n29
bv_n30
knn_n31
cc_n32
QV_n32
dnn_n33
ising_n34
cat_n35
wstate_n36
bwt_n37
qugan_n39
ghz_n40
swap_test_n41
knn_n41
ising_n42
multiplier_n45
square_root_n45
dnn_n51
bwt_n57
square_root_n60
qft_n63
cc_n64
adder_n64
cat_n65
ising_n66
knn_n67
bv_n70
qugan_n71
multiplier_n75
wstate_n76
ghz_n78
swap_test_n83
bwt_n97
ising_n98
QV_n100


In [2]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import grid_coupling, linear_coupling, random_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_QASM_grid_2.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################

means_filename = "benchmark_means_MLFM-R_QASM_grid_2.json"

circ_name_list.remove('QV_n100')  # Remove the specific circuit if it exists

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [6,8]

for num_partitions in num_partition_list:

    for i, circ_name in enumerate(circ_name_list):
        i = i
        circ = bm.get(circ_name)
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True)
            depth = base_graph.depth
            qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
            coupling = grid_coupling(num_partitions)
            network = QuantumNetwork(qpu_info, coupling)
            initial_assignment = set_initial_partitions(network, num_qubits, depth)
            num_qubits = circ.num_qubits

            
            iteration_data = []
            for iteration in range(5):
                try:
                    assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                                                                        initial_assignment,  
                                                                                        qpu_info, 
                                                                                        limit = num_qubits, 
                                                                                        costs = {},
                                                                                        network = network,)
                except Exception as e:
                    print(f"Error during MLFM recursive for {circ_name_list[i]}: {e}")
                    continue
                
                total_time_r = sum(time_list_r)
                min_cost_r = min(cost_list_r)


                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "r_cost":  min_cost_r,
                    "time_r": total_time_r,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 5 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            r_cost_list = [x["r_cost"] for x in iteration_data]
            r_time_list = [x["time_r"] for x in iteration_data]
            

            mean_r_cost = float(np.mean(r_cost_list))
            mean_r_time = float(np.mean(r_time_list))
            
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  Recursive:{mean_r_cost:.3f}")
            print("Mean Times (s):")

            print(f"  Recursive:{mean_r_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_r_cost": mean_r_cost,
                "mean_r_time": mean_r_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 28 qubits, 6 partitions, circuit: adder_n28
Number of layers: 191
Time to coarsen graph: 0.04 seconds
Time to coarsen graph: 0.04 seconds
Time to coarsen graph: 0.07 seconds
Time to coarsen graph: 0.04 seconds
Time to coarsen graph: 0.04 seconds
Finished 10 iterations for:
  # Qubits: 28, # Partitions: 6, circuit: adder_n28
Mean Costs:
  Recursive:11.800
Mean Times (s):
  Recursive:5.817
Starting benchmark for 29 qubits, 6 partitions, circuit: qft_n29
Number of layers: 223
Time to coarsen graph: 0.10 seconds
Time to coarsen graph: 0.10 seconds
Time to coarsen graph: 0.15 seconds
Time to coarsen graph: 0.10 seconds
Time to coarsen graph: 0.10 seconds
Finished 10 iterations for:
  # Qubits: 29, # Partitions: 6, circuit: qft_n29
Mean Costs:
  Recursive:70.000
Mean Times (s):
  Recursive:11.253
Starting benchmark for 30 qubits, 6 partitions, circuit: bv_n30
Number of layers: 43
Time to coarsen graph: 0.03 seconds
Error during MLFM recursive for bv_n30: 40
Time to coa

/Users/ftb123/MLQCP_FM/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ftb123/MLQCP_FM/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Time to coarsen graph: 0.02 seconds
Time to coarsen graph: 0.02 seconds
Time to coarsen graph: 0.03 seconds
Time to coarsen graph: 0.03 seconds
Finished 10 iterations for:
  # Qubits: 31, # Partitions: 6, circuit: knn_n31
Mean Costs:
  Recursive:5.000
Mean Times (s):
  Recursive:6.066
Starting benchmark for 32 qubits, 6 partitions, circuit: cc_n32
Number of layers: 135
Time to coarsen graph: 0.02 seconds
Error during MLFM recursive for cc_n32: 131
Time to coarsen graph: 0.02 seconds
Error during MLFM recursive for cc_n32: 131
Time to coarsen graph: 0.06 seconds
Error during MLFM recursive for cc_n32: 131
Time to coarsen graph: 0.02 seconds
Error during MLFM recursive for cc_n32: 131
Time to coarsen graph: 0.02 seconds
Error during MLFM recursive for cc_n32: 131
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 6, circuit: cc_n32
Mean Costs:
  Recursive:nan
Mean Times (s):
  Recursive:nan
Starting benchmark for 32 qubits, 6 partitions, circuit: QV_n32
Number of layers: 195
Time 

KeyboardInterrupt: 

In [ ]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import grid_coupling, linear_coupling, random_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_QASM_linear_2.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################

means_filename = "benchmark_means_MLFM-R_QASM_linear_2.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [6,8]


for num_partitions in num_partition_list:

    for i, circ_name in enumerate(circ_name_list):
        i = i
        circ = bm.get(circ_name)
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True)
            depth = base_graph.depth
            qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
            coupling = linear_coupling(num_partitions)
            network = QuantumNetwork(qpu_info, coupling)
            initial_assignment = set_initial_partitions(network, num_qubits, depth+2)
            num_qubits = circ.num_qubits

            
            iteration_data = []
            for iteration in range(1):

                assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                                                                    initial_assignment,  
                                                                                    qpu_info, 
                                                                                    limit = num_qubits, 
                                                                                    costs = {},
                                                                                    network = network,)
                
                total_time_r = sum(time_list_r)
                min_cost_r = min(cost_list_r)


                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "r_cost":  min_cost_r,
                    "time_r": total_time_r,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 5 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            r_cost_list = [x["r_cost"] for x in iteration_data]
            r_time_list = [x["time_r"] for x in iteration_data]
            

            mean_r_cost = float(np.mean(r_cost_list))
            mean_r_time = float(np.mean(r_time_list))
            
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  Recursive:{mean_r_cost:.3f}")
            print("Mean Times (s):")

            print(f"  Recursive:{mean_r_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_r_cost": mean_r_cost,
                "mean_r_time": mean_r_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 100 qubits, 6 partitions, circuit: QV_n100
Number of layers: 603
Time to coarsen graph: 2.82 seconds
Finished 10 iterations for:
  # Qubits: 100, # Partitions: 6, circuit: QV_n100
Mean Costs:
  Recursive:10840.000
Mean Times (s):
  Recursive:119.777
Starting benchmark for 100 qubits, 8 partitions, circuit: QV_n100
Number of layers: 603
Time to coarsen graph: 2.94 seconds
Finished 10 iterations for:
  # Qubits: 100, # Partitions: 8, circuit: QV_n100
Mean Costs:
  Recursive:16084.000
Mean Times (s):
  Recursive:177.464
Benchmarking completed. Detailed results saved to benchmark_results_MLFM-R_QASM_linear_2.json
Aggregated means saved to benchmark_means_MLFM-R_QASM_linear_2.json
